In [1]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
#using NearestNeighbors # Can't use KDtree in julia for posterior prediction

In [2]:
Sys.free_memory()/(2^20*1024)

18.924705505371094

In [3]:
#using Pkg
#Pkg.add("RData")

In [4]:
include("../../../util2.j")

colnorm (generic function with 1 method)

In [5]:
# unnecessary packages #

#using Pkg
#Pkg.add("UnicodePlots")
using UnicodePlots   # check the structure of the sparse matrix
using BenchmarkTools

using StatsPlots
using MCMCChains
using PrettyTables
using Dates
using StatsBase

In [6]:
#using Pkg
#Pkg.add("StatsBase");
#using NearestNeighborDescent

In [7]:
using RData
data_cleaned = load("../../data/rawdata/cleaned_data2_expanded.RData", convert = true)["data_cleaned2"];

In [10]:
size(data_cleaned)

(3115934, 28)

In [11]:
countmap(data_cleaned.LC_Type4)

Dict{Float64,Int64} with 7 entries:
  7.0 => 253060
  4.0 => 93157
  2.0 => 25572
  5.0 => 83245
  8.0 => 9496
  6.0 => 904677
  1.0 => 1746727

In [13]:
hold_index_set = load("../../data/rawdata/hold_index_expanded.RData", convert = true);

In [14]:
U_indx = sort(intersect(hold_index_set["hold_ind1"], hold_index_set["hold_ind2"]));
N = size(data_cleaned)[1];
S_indx = setdiff(1:N, U_indx);

In [15]:
# coordinate:
coords = [data_cleaned.scaled_x[S_indx] data_cleaned.scaled_y[S_indx]];
coords_pred = [data_cleaned.scaled_x[U_indx] data_cleaned.scaled_y[U_indx]];
n = size(coords)[1]; m = 10;

In [16]:
#@load "../../data/conj_data/conj_data.jld"

In [17]:
Dates.format(now(), "HH:MM:SS")  

"00:21:16"

In [18]:
# build neighbors #
m = 10;                                                       # number of nearest neighbor                       
NN = BuildNN(coords, m, 1.0, 0.0);                       # build nearest neighbor 
nnIndx_col = vcat(NN.nnIndx, 1:n);                            # the index of columns
nnIndx_row = zeros(Int64, 0);                                               
for i in 2:m
    nnIndx_row = vcat(nnIndx_row, fill(i, i-1))
end
nnIndx_row = vcat(nnIndx_row, repeat((m + 1):n, inner = m), 1:n);  # the index of rows

In [19]:
Dates.format(now(), "HH:MM:SS")  

"05:45:33"

In [20]:
# construct Atilde Dtilde #

using RCall
@rput coords
@rput coords_pred
@rput m
R"""
library("RANN")
nn_mod_ho <- nn2(coords, coords_pred, k = m)
"""
@rget nn_mod_ho
NM = size(coords_pred)[1];
MnnIndxLU = collect(1:m:(NM * m + 1));
MnnIndx = vec(nn_mod_ho[:nn_idx]');
Mnndists = vec(nn_mod_ho[:nn_dists]');

In [21]:
Dates.format(now(), "HH:MM:SS")  

"05:45:44"

In [22]:
Dates.format(now(), "HH:MM:SS")  

"05:45:46"

In [23]:
# cross-validation Preparison
# (a) Randomly split 1:n into K folder 
Random.seed!(321);
kf = 5;
parts = kfoldperm(n,kf);
parts_train = [sort(setdiff(1:n, parts[i])) for i in 1:kf];
parts_test = [sort(parts[i]) for i in 1:kf];

# (b) Build NN index for training data in ith folder
using RCall
CVNN_train = Array{Any,1}(undef, kf);
CVNN_test = Array{Any,1}(undef, kf);
CVnnIndx_col = Array{Any,1}(undef, kf);
CVnnIndx_row = Array{Any,1}(undef, kf);
CVMnnIndxLU = Array{Any,1}(undef, kf); 
CVMnnIndx = Array{Any,1}(undef, kf);
CVMnndists = Array{Any,1}(undef, kf);

@rput coords
@rput S_indx
@rput m
@rput parts_train
@rput parts_test

for i in 1:kf
    
    # Build NN index for training data in ith folder
    CVNN_train[i] = BuildNN(coords[parts_train[i], :], m, 1.0);
    CVnnIndx_col[i] = vcat(CVNN_train[i].nnIndx, 1:(n - length(parts[i])));
    CVnnIndx_row[i] = zeros(Int64, 0);
    for j in 2:m
        CVnnIndx_row[i] = vcat(CVnnIndx_row[i], fill(j, j-1));
    end
    CVnnIndx_row[i] = vcat(CVnnIndx_row[i], 
        repeat((m + 1):(n - length(parts[i])), inner = m), 1:(n - length(parts[i])));
        
    # Build nearest neighbor index for holdout locations
    @rput i
    R"""
    library("RANN")
    nn_mod_ho <- nn2(coords[parts_train[[i]], ], coords[parts_test[[i]], ], k = m)
    """
    @rget nn_mod_ho
    CVNN_test[i] = nn_mod_ho;
    CVMnnIndxLU[i] = collect(1:m:(length(parts[i]) * m + 1));
    CVMnnIndx[i] = vec(nn_mod_ho[:nn_idx]');
    CVMnndists[i] = vec(nn_mod_ho[:nn_dists]');
end   


In [24]:
Dates.format(now(), "HH:MM:SS")  

"23:07:17"

In [26]:
@save "../../data/conj_data/conj_data_expanded.jld" U_indx S_indx N n m NN nnIndx_col nnIndx_row NM MnnIndxLU MnnIndx Mnndists coords coords_pred CVNN_train CVnnIndx_col CVnnIndx_row CVNN_test CVMnnIndxLU CVMnnIndx CVMnndists parts_train parts_test kf